In [74]:
regional_key = 'Landeshauptstadt München'

import sys
sys.path.append('/Users/felix/Documents/thesis')

import pharmada.regkey as rk
import pharmada.geometry as geo
import pharmada.pharmacies as ph
import pharmada.overpass as op
import osm2geojson as o2g
import geopandas as gpd
import requests as req
from shapely.geometry import Point

In [56]:
RegKey = rk.RegKey(regional_key)
AreaGeometry = geo.AreaGeometry(RegKey)

In [57]:
query = fr"""
[out:json];
    
//set up area given by regional_key as search boundary
(
relation({AreaGeometry.osm_id});
map_to_area;
) ->.boundary;

wr(area.boundary)[type=boundary][boundary=postal_code]->.results;

// output full geometry of the results set     
.results out geom qt;
"""

response = op.query_overpass(query)

In [58]:
result = o2g.json2geojson(response)
# Convert the geometry to a GeoDataFrame
geometry = gpd.GeoDataFrame.from_features(result, crs='EPSG:4326')
geometry['postal_code'] = geometry['tags'].apply(lambda tags: tags.get('postal_code'))


In [76]:
test_address = 'Frankenstraße 99, 97078 Würzburg'

# geocode with Nominatim API
url = 'https://nominatim.openstreetmap.org/search'
params = {'q': test_address, 'format': 'json'}
response = req.get(url, params=params).json()

# get coordinates from response
lat = float(response[0]['lat'])
lon = float(response[0]['lon'])

# build point
point = Point(lon, lat)

# build GeoSeries from point and display
gs = gpd.GeoSeries(point)


In [79]:
# reverse geocode the coordinates with Nominatim API
url = f'https://nominatim.openstreetmap.org/reverse?format=json&lat={lat}&lon={lon}'
response = req.get(url).json()

In [80]:
response

{'place_id': 621513,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'node',
 'osm_id': 288529414,
 'lat': '49.8081696',
 'lon': '9.9614443',
 'display_name': 'Bayernstraße, Frankenstraße, Lindleinsmühle, Würzburg, Bayern, 97078, Deutschland',
 'address': {'highway': 'Bayernstraße',
  'road': 'Frankenstraße',
  'suburb': 'Lindleinsmühle',
  'city': 'Würzburg',
  'state': 'Bayern',
  'ISO3166-2-lvl4': 'DE-BY',
  'postcode': '97078',
  'country': 'Deutschland',
  'country_code': 'de'},
 'boundingbox': ['49.8081196', '49.8082196', '9.9613943', '9.9614943']}